In [38]:
from tensorflow.keras.layers import LSTM,Bidirectional,Input,Concatenate,Dense
from tensorflow.keras.models import Model as Model_


class GloGen_Bidirectional_RNN_Encoder(Model_):
    def __init__(self, num_recurrent_neurons=64):
        super(GloGen_Bidirectional_RNN_Encoder, self).__init__()
        self.encoder = Bidirectional(LSTM(num_recurrent_neurons, return_state=True))       

    def call(self, input_x):
        self.encoder_outputs, self.forward_h, self.forward_c, self.backward_h, self.backward_c = self.encoder(input_x)
        hidden_state = Concatenate()([self.forward_h, self.backward_h])
        cell_state = Concatenate()([self.forward_c, self.backward_c])
        return [hidden_state,cell_state]
    
class GloGen_Bidirectional_RNN_Decoder(Model_):
    def __init__(self, num_recurrent_neurons=64):
        super(GloGen_Bidirectional_RNN_Decoder, self).__init__()
        self.decoder = LSTM(num_recurrent_neurons*2, return_sequences=True, return_state=True)
        #self.decoder = Bidirectional(LSTM(num_recurrent_neurons, return_sequences=True, return_state=True)) 
        self.decoder_input=Input(shape=(10, num_recurrent_neurons))
        

    def call(self, encoder_hidden_state,num_recurrent_neurons=64):
        self.decoder_outputs, _, _ = self.decoder(self.decoder_input,encoder_hidden_state)
        self.decoder_dense = Dense(num_recurrent_neurons, activation='softmax')
        decoder_outputs = self.decoder_dense(self.decoder_outputs)
        return decoder_outputs
    
#ref: https://stackoverflow.com/questions/50815354/seq2seq-bidirectional-encoder-decoder-in-keras

In [39]:
import pandas as pd
import numpy as np
direction_csv_file = pd.read_csv("Directions 1.54138969.csv", header=None)
direction_data_array = np.asarray(direction_csv_file)
direction_data_array = np.reshape(direction_data_array[:100], (10, 10, 64))
direction_data_array.shape

(10, 10, 64)

In [40]:
glogen_bidirectional_RNN_Encoder = GloGen_Bidirectional_RNN_Encoder()
output_encoder = glogen_bidirectional_RNN_Encoder(direction_data_array)
output_encoder


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



[<tf.Tensor: shape=(10, 128), dtype=float32, numpy=
 array([[-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -4.4779837e-25],
        [-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -1.4728992e-21],
        [-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -1.0503201e-18],
        ...,
        [-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -3.8445863e-35],
        [-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -7.9705662e-34],
        [-0.0000000e+00, -1.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00, -4.7956878e-33]], dtype=float32)>,
 <tf.Tensor: shape=(10, 128), dtype=float32, numpy=
 array([[-1.0000000e+00, -1.0000000e+01,  1.0000000e+01, ...,
          0.0000000e+00,  0.0000000e+00, -4.4779837e-25],
        [-1.0000000e+00, 

In [41]:
glogen_decoder = GloGen_Bidirectional_RNN_Decoder()
glogen_sparse_output_predictions = glogen_decoder(output_encoder)
print(glogen_sparse_output_predictions.shape)
glogen_sparse_output_predictions

(10, 10, 64)


<tf.Tensor 'dense_2/truediv:0' shape=(10, 10, 64) dtype=float32>